# Add labels to DB
For every _valid_ experiment **from the validation data**, add labels to a separate table.
We use the following labels:
- scaled MS responses for all products
- binary outcome for all products
- major product (A, B, C, or no_product)

We use the scaling factors obtained on the ML data set (2024-04-18).

In [ ]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import pandas as pd

from src.util.db_utils import SynFermDatabaseConnection
from src.definitions import DATA_DIR

In [ ]:
con = SynFermDatabaseConnection()

In [ ]:
# get only valid experiments from DB
data = con.con.execute("SELECT * FROM experiments WHERE exp_nr BETWEEN 100 AND 101 AND (valid NOT LIKE '%ERROR%' OR valid IS NULL)").fetchall()
df = pd.DataFrame(data, columns=[c[1] for c in con.con.execute("PRAGMA table_info(experiments)").fetchall()])
len(df)

In [ ]:
# load scaling factors
scaling_factors = pd.read_csv(DATA_DIR / "scaling-factors_2024-04-18.csv").set_index("product_type")

In [ ]:
# apply scaling to all products
for s in "ABCDEFGH":
    df[f'scaled_{s}'] = df[f'product_{s}_lcms_ratio'] / scaling_factors.loc["A", "factor"]

In [ ]:
# add binary outcome: 1 if product was formed, 0 if not
df["binary_A"] = (df["scaled_A"] > 0).astype(int)
df["binary_B"] = (df["scaled_B"] > 0).astype(int)
df["binary_C"] = (df["scaled_C"] > 0).astype(int)
df["binary_D"] = (df["scaled_D"] > 0).astype(int)
df["binary_E"] = (df["scaled_E"] > 0).astype(int)
df["binary_F"] = (df["scaled_F"] > 0).astype(int)
df["binary_G"] = (df["scaled_G"] > 0).astype(int)
df["binary_H"] = (df["scaled_H"] > 0).astype(float)
df.loc[df["scaled_H"].isna(), "binary_H"] = pd.NA

In [ ]:
df.describe()

In [ ]:
# assign the main product
df["main_product"] = df[["scaled_A", "scaled_B", "scaled_C"]].idxmax(axis=1).str.replace("scaled_", "")
# are there any reactions where neither A,nor B, nor C appear?
df.loc[df[["scaled_A", "scaled_B", "scaled_C"]].sum(axis=1) == 0, "main_product"] = "no_product"
df

In [ ]:
# write to 'labels' table of DB
with con.con:
    con.con.executemany('INSERT INTO labels (experiment_id, scaled_A, scaled_B, scaled_C, scaled_D, scaled_E, scaled_F, scaled_G, scaled_H, binary_A, binary_B, binary_C, binary_D, binary_E, binary_F, binary_G, binary_H, "major_A-C") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);',
                        df[["id", "scaled_A", "scaled_B", "scaled_C", "scaled_D", "scaled_E", "scaled_F", "scaled_G", "scaled_H", "binary_A", "binary_B", "binary_C", "binary_D", "binary_E", "binary_F", "binary_G", "binary_H", "main_product"]].values.tolist()
                        )

In [ ]:
df["binary_A"].mean()